In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [53]:
data = pd.read_feather('data.feather')

/home/jeremie/miniconda/envs/nhis/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [54]:
data.shape

(78132, 112)

In [55]:
data = data.astype('float64')

In [56]:
data.drop(columns=['YEAR','SERIAL','NUMPREC','STRATA','PSU','NHISHID','HHWEIGHT','PERNUM',
                   'NHISPID','HHX','FMX','PX','PERWEIGHT','SAMPWEIGHT','FWEIGHT','SUPP1WT',
                   'ASTATFLG','CSTATFLG','FAMNUMR', # irrelevant for prediction
                   'LANY', # categories make no sense
                   ], inplace=True)

In [57]:
categorical = ['REGION','LIVINGQTR','INTERVWMO','SEX','SEXORIEN','MARSTAT','MARST','MARSTCOHAB',
               'PARENTHERE','RELMOM','MOMED','RELPOP','DADED','RACEA','HISPETH','YRSINUS','USBORN',
               'CITIZEN','RACENEW','INTERVLANG','EDUC','HEADSTAREV','HEADSTARNOW','EMPSTAT',
               'MONTHWRK','USUALFT','EMPSTATWKYR','WORKEV','POORYN','INCFAM07ON','EARNINGS',
               'GOTWELF','GOTNEWELF','GOTSSI','GOTSS','GOTSSDIS','GOTNONSSDIS','GOTDIV','GOTINT',
               'GOTOTHPEN','GOTWAGE','GOTSEMP','GOTCHSUP','GOTSTAMPFAM','OWNERSHIP','LOWRENT',
               'HEALTH','WORKVOLHCSET','WORKVOLHCPAT','NBHDTRUST','PREGNANTNOW','USUALPL','TYPPLSICK',
               'HIUNABLEPAY','HINOTCOVE','ANEMIAYR','CHOLHIGHYR','CRONBRONYR','DIARRHEAYR','EARINFYR',
               'FHEADYR','HYPERTENYR','LIVERCONYR','RALLERGYR','SALLERGYR','SEIZUREYR','ULCERYR',
               'ALCSTAT1','SMOKFREQNOW','CSQTRYYR','MOD10FWK','VIG10FWK','EXERDYR','EXERNOW','DIETDYR',
               'DIETNOW','LANY']

numerical = ['AGE','FAMSIZE','FAMKIDNO','FAMOLDNO','HOURSWRK','HRSLEEP']

missing_map = {'REGION': (8,9),
               'LIVINGQTR': (98,),
               'INTERVWMO': (98,),
               'AGE': (99,),
               'SEXORIEN': (0,8),
               'MARSTAT': (0,99),
               'MARST': (0,99),
               'MARSTCOHAB': (9,),
               'FAMSIZE': (0,99),
               'PARENTHERE': (0,98,99),
               'RELMOM': (0,60,97,98,99),
               'MOMED': (0,97,98,99),
               'RELPOP': (0,60,97,98,99),
               'DADED': (0,97,98,99),
               'RACEA': (900,970,980,990),
               'HISPETH': (90,91,92,93,99),
               'YRSINUS': (0,7,8,9),
               'USBORN': (96,97,98,99),
               'CITIZEN': (7,8,9),
               'RACENEW': (97,98,99),
               'INTERVLANG': (8,9),
               'EDUC': (0,997,998,999),
               'HEADSTAREV': (0,7,8,9),
               'HEADSTARNOW': (0,7,8,9),
               'EMPSTAT': (0,90,97,98,99),
               'HOURSWRK': (0,97,98,99),
               'MONTHWRK': (13,90,97,98,99),
               'USUALFT': (0,7,8,9),
               'EMPSTATWKYR': (0,7,8,9),
               'WORKEV': (0,7,8,9),
               'POORYN': (9,),
               'INCFAM07ON': (96,99),
               'EARNINGS': (0,97,98,99),
               'GOTWELF': (0,70,80,90),
               'GOTNEWELF': (0,7,8,9),
               'GOTSSI': (0,70,80,90),
               'GOTSS': (0,70,80,90),
               'GOTSSDIS': (0,97,98,99),
               'GOTNONSSDIS': (0,70,80,90),
               'GOTDIV': (0,70,80,90),
               'GOTINT': (0,70,80,90),
               'GOTOTHPEN': (0,7,8,9),
               'GOTWAGE': (0,7,8,9),
               'GOTSEMP': (0,7,8,9),
               'GOTCHSUP': (7,8,9),
               'GOTSTAMPFAM': (0,70,80,90),
               'OWNERSHIP': (0,97,98,99),
               'LOWRENT': (0,7,8,9),
               'HEALTH': (0,7,8,9),
               'WORKVOLHCSET': (0,7,8,9),
               'WORKVOLHCPAT': (0,7,8,9),
               'NBHDTRUST': (0,7,8,9),
               'PREGNANTNOW': (0,7,8,9),
               'USUALPL': (0,7,8,9),
               'TYPPLSICK': (0,97,98,99),
               'HIUNABLEPAY': (0,7,8,9),
               'HINOTCOVE': (0,7,8,9),
               'ANEMIAYR': (0,7,8,9),
               'CHOLHIGHYR': (0,7,8,9),
               'CRONBRONYR': (0,7,8,9),
               'DIARRHEAYR': (0,7,8,9),
               'EARINFYR': (0,7,8,9),
               'FHEADYR': (0,7,8,9),
               'HYPERTENYR': (0,7,8,9),
               'LIVERCONYR': (0,7,8,9),
               'RALLERGYR': (0,7,8,9),
               'SALLERGYR': (0,7,8,9),
               'SEIZUREYR': (0,7,8,9),
               'ULCERYR': (0,7,8,9),
               'ALCSTAT1': (0,9),
               'SMOKFREQNOW': (0,7,8,9),
               'CSQTRYYR': (0,7,8,9),
               'MOD10FWK': (0,97,98,99),
               'VIG10FWK': (0,97,98,99),
               'EXERDYR': (0,7,8,9),
               'EXERNOW': (0,7,8,9),
               'DIETDYR': (0,7,8,9),
               'DIETNOW': (0,7,8,9),
               'HRSLEEP': (0,25,97,98,99),
               'LANY': (20,22)}

In [58]:
data['WORKEV']

0        2.0
1        0.0
2        0.0
3        2.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       1.0
16       0.0
17       2.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       2.0
27       2.0
28       0.0
29       0.0
        ... 
78102    0.0
78103    2.0
78104    0.0
78105    0.0
78106    0.0
78107    2.0
78108    0.0
78109    0.0
78110    0.0
78111    0.0
78112    0.0
78113    2.0
78114    0.0
78115    2.0
78116    0.0
78117    0.0
78118    0.0
78119    0.0
78120    0.0
78121    2.0
78122    0.0
78123    2.0
78124    0.0
78125    2.0
78126    0.0
78127    2.0
78128    0.0
78129    0.0
78130    0.0
78131    0.0
Name: WORKEV, Length: 78132, dtype: float64

In [59]:
for feature in categorical:
    if feature in missing_map:
        for val in missing_map[feature]:
            data[feature] = data[feature].apply(lambda x: x if x != val else np.nan)

In [42]:
for feature in numerical:
    if feature in missing_map:
        for val in missing_map[feature]:
            data[feature] = data[feature].apply(lambda x: x if x != val else np.nan)

In [127]:
X = data[categorical + numerical]

all_targets = ['AEFFORT','AFEELINT1MO','AHOPELESS','ANERVOUS','ARESTLESS',
           'ASAD','AWORTHLESS','DEPFREQ','DEPRX','DEPFEELEVL']

k6 = ['AEFFORT','ASAD','ANERVOUS','ARESTLESS','AHOPELESS','AWORTHLESS']

y = data[k6]

mask = np.full((data.shape[0]), True, dtype=bool)
for f in k6:
    for v in (6,7,8,9):
        # 6: not applicable (child or not selected)
        # 7,8,9: missing
        mask = np.logical_and(mask, y[f] != v)

y = y[mask].sum(axis=1)
X = X[mask]

In [131]:
# drop full missing values features
full_missing = (X.count() == 0).sum()

11